In [17]:
import os
import json
from dotenv import load_dotenv
load_dotenv('/Users/joe.constantino/Desktop/playground/tableau_langchain/.env')

#base langchain library imports
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, create_tool_calling_agent, AgentExecutor

#tableau_langchain imports
from utilities.tableau.readMetadata import read
from utilities.tableau.setPrompt import instantiate_prompt
from tools.tableau.getDataTool import get_data

In [21]:
#read in the environment varibales to query a Tableau Published Data Source from a target Tableau Cloud Site or Server Instance
datasource = os.getenv('DATASOURCE_LUID')
read_metadata_url = os.getenv('READ_METADATA')
query_datasource = os.getenv('QUERY_DATASOURCE')
token = os.getenv('AUTH_TOKEN')

In [ ]:
## Use the read module to pull in the necessary metadata from your target Tableau Published Data Source. This will module also augment the
## base metadata provided by the read_metadata endpoint by removing unneccesary fields and adding sample field values to each string field.
metadata = read(read_url=read_metadata_url, datasource_luid=datasource, auth_secret=token)

# use the instaniate_prompt module to add the data model metadata to the nlq_to_vds prompt
instantiate_prompt(metadata = metadata)

In [ ]:
# Initialize an LLM and an agent with the get_data tool to answer natural language questions with Tableau Published Datasources
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
tools = [get_data]

tableauHeadlessAgent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS, # Use OpenAI's function calling
    verbose = True
)

# Run the agent
response = tableauHeadlessAgent.invoke("total sales per region")
print(response)